# 04-02 WebSocket 实时通信

实现双向实时通信。

## 1. WebSocket 服务器

In [ ]:
import { WebSocketServer } from 'ws';

const wss = new WebSocketServer({ port: 8080 });

wss.on('connection', (ws) => {
  console.log('Client connected');
  
  // 接收消息
  ws.on('message', (data) => {
    console.log('Received:', data.toString());
    
    // 广播给所有客户端
    wss.clients.forEach((client) => {
      if (client.readyState === 1) {
        client.send(`Echo: ${data}`);
      }
    });
  });
  
  ws.on('close', () => {
    console.log('Client disconnected');
  });
  
  // 发送欢迎消息
  ws.send(JSON.stringify({ type: 'welcome', message: 'Connected!' }));
});

console.log('WebSocket server running on ws://localhost:8080');

## 2. 心跳机制

In [ ]:
// 心跳保持连接
function heartbeat(ws) {
  ws.isAlive = true;
}

wss.on('connection', (ws) => {
  ws.isAlive = true;
  
  ws.on('pong', () => heartbeat(ws));
  
  // 定期检测
  const interval = setInterval(() => {
    wss.clients.forEach((ws) => {
      if (ws.isAlive === false) return ws.terminate();
      
      ws.isAlive = false;
      ws.ping();
    });
  }, 30000);
  
  ws.on('close', () => {
    clearInterval(interval);
  });
});

## 练习

1. 实现聊天室
2. 添加用户认证
3. 实现消息持久化